In [64]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import requests as req
import json
import ta 
from datetime import datetime
import datetime as datetimee
from dateutil.relativedelta import relativedelta
import warnings
import time as tm
warnings.filterwarnings("ignore")

### Data - CurrentData

In [65]:
CurrentData = pd.read_csv('D:\\Users\\amit\\Desktop\\מסחר\\Python\\QQQ5M2021_8-2023_7.csv')

### Add Stoch-RSI, PChange, Daily Gap, Daily Range (Range Require Gap)

In [66]:
# Define input values (CurrentData)
smoothK = 3
lengthRSI = 6
lengthStoch = 11
# Calculate RSI and Stochastic RSI
CurrentData['RSI'] = ta.momentum.RSIIndicator(CurrentData['ClosePrice'], lengthRSI).rsi()
rsi1 = CurrentData['RSI']
CurrentData['StochRSI'] = ta.momentum.StochasticOscillator(CurrentData['RSI'], CurrentData['RSI'], CurrentData['RSI'], window=lengthStoch, smooth_window=smoothK).stoch()
# Calculate the smoothed K value
CurrentData['StochRSI'+str(lengthRSI)+'_'+str(lengthStoch)] = round(CurrentData['StochRSI'].rolling(window=smoothK).mean(),2)
CurrentData.drop(columns=['RSI','StochRSI'], inplace=True) #drop extra columns

CurrentData['CandleRange'] = round(CurrentData.Highp - CurrentData.Lowp,2)  #price range of candle $
CurrentData['PChange'] = round(CurrentData['CandleRange']/CurrentData['OpenPrice']*100,2) #create PChange to filter for first candles

gapadd = CurrentData.groupby('Date').Time.max().to_frame().reset_index().merge(CurrentData.groupby('Date').Time.min().to_frame().reset_index(), on='Date')
gapadd.rename(columns={'Time_x': 'CloseTime', 'Time_y': 'OpenTime'}, inplace=True)
gapadd = CurrentData.merge(gapadd, on = 'Date')
gapadd = CurrentData[(gapadd.Time==gapadd.CloseTime) | (gapadd.Time==gapadd.OpenTime)].sort_values(['Date','Time'])
gapadd['DayGap'] = None
for i in range(gapadd.shape[0]):
    if i==0:
        gapadd['DayGap'].iloc[i] = 0
    elif i//2==i/2:
        gapadd['DayGap'].iloc[i] = round((gapadd.OpenPrice.iloc[i]-gapadd.ClosePrice.iloc[i-1])/gapadd.ClosePrice.iloc[i-1]*100, 2)
    else:
        gapadd['DayGap'].iloc[i] = gapadd['DayGap'].iloc[i-1]
CurrentData = CurrentData.merge(gapadd[gapadd.Time=='16:30:00'][['Date','DayGap']], on='Date') 
CurrentData.DayGap = CurrentData.DayGap.astype(float)

dayrang = CurrentData.groupby('Date').Highp.max().to_frame().reset_index().merge(CurrentData.groupby('Date').Lowp.min().to_frame().reset_index(), on='Date')
dayrang.rename(columns={'Highp_x': 'DayHigh', 'Lowp_x': 'DayLow'}, inplace=True)
dayrang['DayRange'] = round(dayrang['Highp']-dayrang['Lowp'],2)
CurrentData = CurrentData.merge(dayrang[['Date','DayRange']], on='Date') 
CurrentData.DayGap = CurrentData.DayGap.astype(float)
CurrentData

,ticker,OpenPrice,ClosePrice,Highp,Lowp,Volume,VWAP,Time,Date,StochRSI6_11,CandleRange,PChange,DayGap,DayRange
0,QQQ,367.6700,367.960,368.2200,367.5900,884521.0,367.9129,16:30:00,2021-08-16,NaN,0.63,0.17,0.00,5.32
1,QQQ,367.9550,368.230,368.3500,367.8999,640912.0,368.1458,16:35:00,2021-08-16,NaN,0.45,0.12,0.00,5.32
2,QQQ,368.2600,367.980,368.4500,367.9100,534271.0,368.1916,16:40:00,2021-08-16,NaN,0.54,0.15,0.00,5.32
3,QQQ,367.9850,367.550,368.1750,367.3500,806842.0,367.7190,16:45:00,2021-08-16,NaN,0.82,0.22,0.00,5.32
4,QQQ,367.5400,366.438,367.5700,366.3800,822006.0,366.9206,16:50:00,2021-08-16,NaN,1.19,0.32,0.00,5.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39045,QQQ,369.2550,369.520,369.6300,369.2550,436500.0,369.5017,22:35:00,2023-08-14,80.93,0.38,0.10,-0.21,5.77
39046,QQQ,369.5100,369.950,369.9900,369.4900,1178629.0,369.6131,22:40:00,2023-08-14,90.65,0.50,0.14,-0.21,5.77
39047,QQQ,369.9339,369.720,369.9585,369.5939,579281.0,369.7467,22:45:00,2023-08-14,82.28,0.36,0.10,-0.21,5.77
39048,QQQ,369.7200,369.730,369.8200,369.4900,1155874.0,369.0401,22:50:00,2023-08-14,64.95,0.33,0.09,-0.21,5.77


### Graph and RSI by Date

In [96]:
# Create the candlestick chart using plotly, Change Sample Data accordingly and specify Date/s
CandCharData = CurrentData #Sample Data
GraphDate = str(input('Date to show (YYYY-MM-DD):')).split(',')
CandCharData = CandCharData[CandCharData.Date.isin(GraphDate)]      #Filter for specified date
fig = go.Figure(data=[go.Candlestick(x=CandCharData['Time'],
                open=CandCharData['OpenPrice'],
                high=CandCharData['Highp'],
                low=CandCharData['Lowp'],
                close=CandCharData['ClosePrice'])])

fig.update_layout(
    title=str(CandCharData.ticker.unique())+', '+str(CandCharData.Date.unique()),
    xaxis_title='Time',
    yaxis_title='Price', 
    yaxis=dict(tickangle=0),
    xaxis_rangeslider_visible=False)
fig.show()

fig = px.line(CandCharData, x='Time', y='StochRSI6_11')
fig.update_layout(width=985, height=300)  # You can adjust width and height as needed
fig.update_layout(shapes=[
    dict(type="line", x0=CandCharData['Time'].iloc[0], x1=CandCharData['Time'].iloc[-1], y0=20, y1=20,
         line=dict(color="black", width=1, dash="dot")),
    dict(type="line", x0=CandCharData['Time'].iloc[0], x1=CandCharData['Time'].iloc[-1], y0=80, y1=80,
         line=dict(color="black", width=1, dash="dot"))])
fig.show()

Date to show (YYYY-MM-DD):2023-08-10


In [169]:
#### Data Collection for opening trades when Stoch-RSI is above or below 20/80 for reversals ####
DayStartTime = '17:00:00'
RSITest = CurrentData[CurrentData.Time>=DayStartTime] #filter day start time
RSITest = RSITest[RSITest.Date=='2023-08-10'] #filter day for loop build process
RSITestData = {'Ticker':[],'Position': [], 'Date': [],'EntryTime': [], 'CloseTime': [],'TimeToPeak': [], 'RMax': [], 'PriceToPeak': [],'PChangePeak': [],'StopSize': [], 'RSICross': [], 'DayTNum': []} #empty results dictionary to create dataframe later
def TimeToPeak (PeakTime, EntryTime):
    time_obj1 = datetime.strptime(PeakTime, '%H:%M:%S').time()
    time_obj2 = datetime.strptime(EntryTime, '%H:%M:%S').time()
    time_difference = datetime.combine(datetime.today(), time_obj1) - datetime.combine(datetime.today(), time_obj2)
    return(str(time_difference)[:4])
xtest = 0
yest = 0
dailyTNum = 1
Engulfing_candles = 0
CurrHigh = 0
CurrLow = 0
RSICross = 'No' #variable to know if RSI crossed to other side
CurrDate = ''
PeakTime = '' 
CrossIndex = 0 #variable to know if RSI came inside to 65 or 35 to avoid extra trade logs
for j in range (RSITest.head(1).index[0], RSITest.head(1).index[0]+RSITest.shape[0]): #from first index till end of filtering
    yest += 1
    if RSITest.loc[j].Time == DayStartTime:
        CurrDate = RSITest.loc[j].Date
        dailyTNum = 1
    if (RSITest.loc[j].StochRSI6_11>80) and (j > CrossIndex):
        CrossIndex = 0
        if CurrHigh == 0:  #get first values for highs/lows
            CurrHigh = RSITest.loc[j].Highp 
            CurrLow = RSITest.loc[j].Lowp
        elif (RSITest.loc[j].Lowp < CurrLow) and (RSITest.loc[j].Highp >= CurrHigh):
            Engulfing_candles += 1 #count candles where we dont know if it was an entry with new high or immediate loss           
        elif RSITest.loc[j].Highp >= CurrHigh: #update high and low
            CurrHigh = RSITest.loc[j].Highp
            CurrLow = RSITest.loc[j].Lowp #update to higher high and higher low for entry
        elif (RSITest.loc[j].Lowp < CurrLow) and (RSITest.loc[j-1].Date == CurrDate): #open trade, make sure its in same day
            CrossIndex = 0
            RSITestData['DayTNum'].append(dailyTNum)
            dailyTNum += 1
            RSITestData['EntryTime'].append(RSITest.loc[j].Time)
            RSITestData['Ticker'].append(RSITest.loc[j].ticker)  
            RSITestData['Date'].append(RSITest.loc[j].Date)  
            RSITestData['StopSize'].append(CurrHigh-CurrLow-0.01)
            RSITestData['Position'].append('Short')
            StopSize = CurrHigh-CurrLow-0.01 #variables for trade close calculations
            EntryPrice = CurrLow-0.01
            EntryTime = RSITest.loc[j].Time
            PeakTime = RSITest.loc[j].Time
            for k in range (j, RSITest.head(1).index[0]+RSITest.shape[0]): 
                xtest += 1
                if (not CurrDate==RSITest.loc[k].Date) or (RSITest.loc[k].Time=='22:55:00'):  #avoid day change 
                    RSITestData['PriceToPeak'].append(round(EntryPrice-CurrLow,2)) #Close trade
                    RSITestData['RMax'].append(round((EntryPrice-CurrLow)/StopSize,2))
                    RSITestData['PChangePeak'].append(round((EntryPrice-CurrLow)/EntryPrice*100,2))              
                    RSITestData['RSICross'].append(RSICross)
                    RSITestData['TimeToPeak'].append(TimeToPeak(PeakTime,EntryTime))
                    if RSITest.loc[k].Time=='22:55:00':
                        RSITestData['CloseTime'].append(RSITest.loc[k].Time) 
                    else:
                        RSITestData['CloseTime'].append(RSITest.loc[k-1].Time)
                    CurrHigh = 0 ; CurrLow = 0 ; RSICross = 'No' ; PeakTime = ''      
                    break
                if (CrossIndex==0) and (RSITest.loc[k].StochRSI6_11 < 65):
                    CrossIndex = k
                if RSITest.loc[k].StochRSI6_11 < 20:
                    RSICross = 'Yes'
                if RSITest.loc[k].Highp > CurrHigh:
                    RSITestData['PriceToPeak'].append(round(EntryPrice-CurrLow,2)) #Close trade
                    RSITestData['RMax'].append(round((EntryPrice-CurrLow)/StopSize,2))
                    RSITestData['PChangePeak'].append(round((EntryPrice-CurrLow)/EntryPrice*100,2))
                    RSITestData['CloseTime'].append(RSITest.loc[k].Time) 
                    RSITestData['RSICross'].append(RSICross)
                    RSITestData['TimeToPeak'].append(TimeToPeak(PeakTime,EntryTime))
                    CurrHigh = 0 ; CurrLow = 0 ; RSICross = 'No' ; PeakTime = ''      
                    break
                if RSITest.loc[k].Lowp < CurrLow:
                    CurrLow = RSITest.loc[k].Lowp
                    PeakTime = RSITest.loc[k].Time    
    if (RSITest.loc[j].StochRSI6_11<20) and (j > CrossIndex):
        CrossIndex = 0
        if CurrHigh == 0:  #get first values for highs/lows
            CurrHigh = RSITest.loc[j].Highp 
            CurrLow = RSITest.loc[j].Lowp
        elif (RSITest.loc[j].Lowp < CurrLow) and (RSITest.loc[j].Highp >= CurrHigh):
            Engulfing_candles += 1 #count candles where we dont know if it was an entry with new high or immediate loss 
        elif RSITest.loc[j].Lowp <= CurrLow: #update high and low
            CurrHigh = RSITest.loc[j].Highp
            CurrLow = RSITest.loc[j].Lowp #update to lower low and lower high for entry
        elif (RSITest.loc[j].Highp > CurrHigh) and (RSITest.loc[j-1].Date == CurrDate): #open trade, make sure its in same day
            CrossIndex = 0
            RSITestData['DayTNum'].append(dailyTNum)
            dailyTNum += 1
            RSITestData['EntryTime'].append(RSITest.loc[j].Time)
            RSITestData['Ticker'].append(RSITest.loc[j].ticker)  
            RSITestData['Date'].append(RSITest.loc[j].Date)  
            RSITestData['StopSize'].append(CurrHigh+0.01-CurrLow)
            RSITestData['Position'].append('Long')
            StopSize = CurrHigh+0.01-CurrLow #variables for trade close calculations
            EntryPrice = CurrHigh+0.01
            EntryTime = RSITest.loc[j].Time
            PeakTime = RSITest.loc[j].Time
            for k in range (j, RSITest.head(1).index[0]+RSITest.shape[0]): 
                if (not CurrDate==RSITest.loc[k].Date) or (RSITest.loc[k].Time=='22:55:00'):  #avoid day change  
                    RSITestData['PriceToPeak'].append(round(CurrHigh-EntryPrice,2)) #Close trade
                    RSITestData['RMax'].append(round((CurrHigh-EntryPrice)/StopSize,2))
                    RSITestData['PChangePeak'].append(round((CurrHigh-EntryPrice)/EntryPrice*100,2))
                    RSITestData['RSICross'].append(RSICross)
                    RSITestData['TimeToPeak'].append(TimeToPeak(PeakTime,EntryTime))
                    if RSITest.loc[k].Time=='22:55:00':
                        RSITestData['CloseTime'].append(RSITest.loc[k].Time) 
                    else:
                        RSITestData['CloseTime'].append(RSITest.loc[k-1].Time)
                    CurrHigh = 0 ; CurrLow = 0 ; RSICross = 'No' ; PeakTime = ''      
                    break
                if (CrossIndex==0) and (RSITest.loc[k].StochRSI6_11 > 35):
                    CrossIndex = k
                if RSITest.loc[k].StochRSI6_11 > 80:
                    RSICross = 'Yes'
                if RSITest.loc[k].Lowp < CurrLow:
                    RSITestData['PriceToPeak'].append(round(CurrHigh-EntryPrice,2)) #Close trade
                    RSITestData['RMax'].append(round((CurrHigh-EntryPrice)/StopSize,2))
                    RSITestData['PChangePeak'].append(round((CurrHigh-EntryPrice)/EntryPrice*100,2))
                    RSITestData['CloseTime'].append(RSITest.loc[k].Time)
                    RSITestData['RSICross'].append(RSICross)
                    RSITestData['TimeToPeak'].append(TimeToPeak(PeakTime,EntryTime))
                    CurrHigh = 0 ; CurrLow = 0 ; RSICross = 'No' ; PeakTime = ''      
                    break
                if RSITest.loc[k].Highp > CurrHigh:
                    CurrHigh = RSITest.loc[k].Highp
                    PeakTime = RSITest.loc[k].Time
RSITestRes = pd.DataFrame(RSITestData).sort_values(['Date','EntryTime'], ascending=True)
RSITestRes

,Ticker,Position,Date,EntryTime,CloseTime,TimeToPeak,RMax,PriceToPeak,PChangePeak,StopSize,RSICross,DayTNum
0,QQQ,Short,2023-08-10,17:05:00,22:55:00,3:50,16.39,6.88,1.84,0.42,Yes,1
1,QQQ,Long,2023-08-10,17:55:00,18:00:00,0:00,0.14,0.15,0.04,1.06,No,2
2,QQQ,Long,2023-08-10,18:10:00,18:20:00,0:00,0.00,0.00,0.00,1.07,No,3
3,QQQ,Short,2023-08-10,18:40:00,22:55:00,2:15,10.13,4.15,1.12,0.41,Yes,4
4,QQQ,Long,2023-08-10,20:05:00,20:20:00,0:00,0.33,0.18,0.05,0.55,No,5
5,QQQ,Short,2023-08-10,21:25:00,21:50:00,0:15,0.43,0.31,0.08,0.72,No,6
6,QQQ,Short,2023-08-10,21:35:00,21:45:00,0:05,0.57,0.30,0.08,0.53,No,7
